# Authentic Pokemon Card Classifier

In [155]:
import os
import numpy as np 
import torch 
import torch.nn as nn 
from torchvision.transforms import transforms 
from torch.utils.data import DataLoader 
from torch.optim import Adam
from torch.optim import SGD
from torch.autograd import Variable
import torchvision
from torch.utils.data import Dataset
import pandas as pd
from skimage import io

Note: for optimizer, test on Adam, but perhaps try SGD as the optimizer as it may generalize better in certain cases. 

In [16]:
# device selection, much better to train on cuda
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Data Preprocessing

In [126]:
transformer = transforms.Compose([
    # ensure that all data is 256x256 input
    transforms.ToTensor(),
    transforms.Resize((256, 256)),
    # increase variation, increase # of unique images by a factor of 2
    transforms.RandomHorizontalFlip(),
    
    transforms.Normalize([0.5, 0.5, 0.5], 
                        [0.5, 0.5, 0.5])
])

#transformer = transforms.ToTensor()

### Hyeperparameters

In [96]:
batch_size = 16
learning_rate = 0.001

# 0: Fake, 1: Real
num_classes = 2
num_epochs = 10

### Datasets and Data Loaders

In [162]:
class RealAndFakeCards(Dataset):
    def __init__(self, csv_file, root_dir, ds_class, transform=None):
        # read in CSV file, with id and label. 
        csv_path = os.path.join(root_dir, csv_file)
        self.annotations = pd.read_csv(csv_path)
        self.root_dir = root_dir
        self.transform = transform
        self.ds_class = ds_class
    
    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, i):
        directory = os.path.join(self.root_dir, self.ds_class)
        img_path = directory + '/' + '{}.jpg'.format(self.annotations.iloc[i, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[i, 1]))

        if self.transform:
            image = self.transform(image)

        return image, y_label

In [163]:
# datasets
train_ds = RealAndFakeCards(csv_file='train_labels.csv', root_dir='pcard_dataset', ds_class='train', transform=transformer)
test_ds = RealAndFakeCards(csv_file='test_labels.csv', root_dir='pcard_dataset', ds_class='test', transform=transformer)

In [164]:
print(test_ds[0])

(tensor([[[0.3647, 0.3647, 0.3647,  ..., 0.4824, 0.4824, 0.4824],
         [0.3647, 0.3725, 0.3725,  ..., 0.4824, 0.4824, 0.4902],
         [0.3804, 0.3804, 0.3804,  ..., 0.4902, 0.4902, 0.4902],
         ...,
         [0.2706, 0.2706, 0.2784,  ..., 0.4118, 0.4118, 0.4118],
         [0.2706, 0.2706, 0.2706,  ..., 0.4118, 0.4039, 0.4039],
         [0.2706, 0.2706, 0.2706,  ..., 0.4039, 0.4039, 0.4039]],

        [[0.3490, 0.3490, 0.3490,  ..., 0.4667, 0.4667, 0.4667],
         [0.3490, 0.3569, 0.3569,  ..., 0.4667, 0.4667, 0.4745],
         [0.3647, 0.3647, 0.3647,  ..., 0.4745, 0.4745, 0.4745],
         ...,
         [0.2627, 0.2627, 0.2706,  ..., 0.4039, 0.4039, 0.4039],
         [0.2627, 0.2627, 0.2627,  ..., 0.4039, 0.3961, 0.3961],
         [0.2627, 0.2627, 0.2627,  ..., 0.3961, 0.3961, 0.3961]],

        [[0.3569, 0.3569, 0.3569,  ..., 0.4745, 0.4745, 0.4745],
         [0.3569, 0.3647, 0.3647,  ..., 0.4745, 0.4745, 0.4824],
         [0.3725, 0.3725, 0.3725,  ..., 0.4824, 0.4824, 0

In [130]:
# data loaders
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dl = DataLoader(test_ds, batch_size=batch_size, shuffle=True)

### CNN Model

In [131]:
classes = ['0', '1']

In [132]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=num_classes):
        super(ConvNet, self).__init__()

        # output after convolution filter = 
        # ((256 - 3 + 2(1))/1) + 1

        # input shape = (batch_size=16, rgb_channels=3, h=256, w=256)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # shape = (16, 12, 256, 256)
        self.bn1 = nn.BatchNorm2d(num_features=12)
        self.relu1 = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=3, stride=1, padding=1)
        self.relu2 = nn.ReLU()

        self.conv3 = nn.Conv2d(in_channels=20, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(num_features=32)
        self.relu3 = nn.ReLU()

        self.fc = nn.Linear(in_features=32*128*128, out_features=num_classes)
    
    def forward(self, x):
        output = self.conv1(x)
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.pool(output)

        output = self.conv2(output)
        output = self.relu2(output)
        
        output = self.conv3(output)
        output = self.bn3(output)
        output = self.relu3(output)

        output = output.view(-1, 32*128*128)

        output = self.fc(output)
        return output

In [133]:
model = ConvNet(num_classes=len(classes)).to(device)

### Training Loop

In [156]:
# loss and optimizer
#optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=0.00) # Adam
optimizer = SGD(model.parameters(), lr=learning_rate) #SGD
loss_function = nn.CrossEntropyLoss()

In [157]:
# number of training data and test data, as per dataset documentation
train_count = 373
test_count = 78

In [160]:
for epoch in range(num_epochs):
    print("Epoch #{}".format(epoch))
    for i, (images, labels) in enumerate(train_dl):
        images = images.to(device)
        labels = labels.to(device)

        # forward pass
        outputs = model(images)
        loss = loss_function(outputs, labels)

        # back propogation 
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

print("Finished Training")

Epoch #0
Epoch #1
Epoch #2
Epoch #3
Epoch #4
Epoch #5
Epoch #6
Epoch #7
Epoch #8
Epoch #9
Finished Training


### Test

In [161]:
test_accuracy = 0.0
with torch.no_grad():
    for i, (images, labels) in enumerate(test_dl):
        images = images.to(device)
        labels = labels.to(device)
        
        outputs=model(images)
        _, prediction = torch.max(outputs, 1)
        test_accuracy += int(torch.sum(prediction==labels.data))
    
    test_accuracy=test_accuracy/test_count
    
    test_accuracy *= 100
print('{} percent accurate.'.format(test_accuracy))

94.87179487179486 percent accurate.
